In [ ]:
!pip install requests
!pip install selenium

In [ ]:
import base64
import requests
import datetime
from urllib.parse import urlencode
from urllib.parse import urlparse
import webbrowser
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import config
import json

In [ ]:
class SpotifyAPI():
    access_token = None
    access_token_expires = None
    access_token_did_expire = True
    client_id = None
    client_secret = None
    did_perform_grant_flow_auth = False
    authorize_url = "https://accounts.spotify.com/authorize"
    scope = None
    headers = None
    version = "v1"
    
    callback_url = None
    parts_of_callback_url = None
    
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def perform_auth_grant_flow(self):
        client_id = self.client_id
        authorize_url = self.authorize_url
        redirect_uri = "http://localhost:8888/callback"
        authorize_url = self.authorize_url
        scope = self.scope
        if scope is None:
            raise Exception("lack of scope, set permissions scope")
        
        parameters = urlencode({"client_id": client_id, "redirect_uri": redirect_uri, "response_type": "token", 
                                "scope": scope})
        request = requests.get(f"{authorize_url}?{parameters}")
        if request.status_code in range(200,299):
            driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")
            driver.get(f"{authorize_url}?{parameters}")
            try:
                WebDriverWait(driver, 60).until(EC.url_contains("access"))
            finally:
                url = driver.current_url
                driver.quit();
                self.callback_url = url
        else:
            raise Exception("failed to request")
    
    def get_callback_url(self):
        return self.callback_url
    
    def parse_redirected_url(self):
        url = self.callback_url
        
        if 'error' in url:
            raise Exception("user denied access")
            
        query = urlparse(url)[5]
        parts_of_callback_url = query.split('&')
    
        for i in range(len(parts_of_callback_url)):
            parts_of_callback_url[i] = parts_of_callback_url[i].split('=')[1]
        self.parts_of_callback_url = parts_of_callback_url
        
    def get_parts_of_redirected_url(self):
        return self.parts_of_callback_url
    
    def set_access_token_and_expire(self):
        self.access_token = self.parts_of_callback_url[0]
        expires_in_seconds = int(self.parts_of_callback_url[2])
        current_time = datetime.datetime.now()
        expires = current_time + datetime.timedelta(seconds=expires_in_seconds)
        self.token_expires = expires
        self.access_token_did_expire = expires < current_time
    
    def get_access_token(self):
        return self.access_token
    
    def get_expires(self):
        return self.token_expires
    
    ### different scopes must be space separated
    def set_scope(self, scope_string):
        self.scope = scope_string
        
    def get_scope(self):
        return self.scope
    
    def set_headers(self, access_token):
        self.headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
        
    def get_headers(self):
        return self.headers
    
    def get_request(self, endpoint):
        headers = self.headers
        
        if self.headers is None:
            raise Exception("didn't set headers!")
            
        r = requests.get(endpoint, headers=headers)
        
        if r.status_code == 204:
            raise Exception("No content in request")
            
        return r
    
    def get_current_user_playlists(self):
        
        if "playlist-read-private" not in self.scope:
            raise Exception("didn't find playlist-read-private in scope")
        
        version = self.version
        endpoint = f"https://api.spotify.com/{version}/me/playlists?limit=50"
        r = self.get_request(endpoint)
        
        return r.json()
    
    def get_playlists_ids(self, playlists_json):
        ids = []
        for item in playlists_json['items']:
            ids.append(item['id'])
        return ids
    
    def get_playlist_tracks(self, playlist_id):
        version = self.version
        endpoint = f"https://api.spotify.com/{version}/playlists/{playlist_id}/tracks"
        r = self.get_request(endpoint)
        return r.json()
    
    def get_tracks_artists_ids(self, tracks_json):
        ids = []
        for item in tracks_json['items']:
            ids.append(item['track']['artists'][0]['id'])
        return ids
    
    def get_artist(self, artist_id):
        version= self.version
        endpoint = f"https://api.spotify.com/{version}/artists/{artist_id}"
        r = self.get_request(endpoint)
        return r.json()
    
    #this way artists will repeat however, one artist mean one song because we can only get song genre
    #by getting artist data  
    def get_artists_name_and_genres(self, artists_ids):
        artists = []
        for item in artists_ids:
            artist_dict = {}
            artist = self.get_artist(item)
            
            if not artist['genres']:
                continue
            
            artist_dict[f"{artist['name']}"] = artist['genres']
            artists.append(artist_dict)
        return artists
    
    def get_all_artists_from_playlists(self, playlist_ids):
        artists_big_list = []
        for playlist_id in playlist_ids:
            tracks_json = self.get_playlist_tracks(playlist_id)
            artists_ids = self.get_tracks_artists_ids(tracks_json)
            artists = self.get_artists_name_and_genres(artists_ids)
            artists_big_list.append(artists)
        return artists_big_list